In [ ]:
# Inports

# plotting libraries
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
sns.set()
# Define figure sizes
plt.rcParams.update({'figure.figsize': (8, 5), 'figure.dpi': 120})

# Data management libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from dateutil.parser import parse 

# Machine Learning libraries
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_predict
import scipy as sp

# Others
import math
from mltools import forecast_tools as FT
import scipy.stats as st

# Non linear models
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.neural_network import  MLPRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from mltools import model_tools as MT
from mltools import regression_tools as RT
# from neuralsens import partial_derivatives as ns

# Load Data

In [ ]:
# Import data

df = pd.read_csv('UnemploymentSpain.dat', sep = '\t')
dates = df['DATE']
df.drop(columns=['DATE'], inplace=True)
df.head()

In [ ]:
# Plotting a time series

fig, ax = plt.subplots()
for col in df.columns.values.tolist():
    ax.plot(col, data=df, label=col, alpha=0.8)
ax.set(title='Time series data', ylabel='Value')
plt.legend()
plt.show()

+ Se puede observar que no es estacionaria en media ni en varianza

In [ ]:
# Load time series values

df_ts = df[['TOTAL']]
df_ts

# SARIMA

# Identification process

In [ ]:
# ACF and PACF of the time series -> identify significant lags and order

plt.figure(figsize=[15,15])
FT.ts_display(df_ts)

# No estacionaria en media (Plot y ACF)
# Se observa cierta estacionalidad en PACF, no podemos identificar estacionalidad a simple vista

+ El decrecimiento lento en ACF podria indicar que se trata de una serie integrada
+ De nuevo observamos que no es estacionaria en media ni varianza
+ Se puede identificar estacionalidad aunque no podemos determinar de forma clara su periodo

#  Stabilize the variance

In [ ]:
# Box-cox transformation
lmbda = FT.boxcox_lambda_plot(df_ts, window_width=12)

# No estacionaria en varianza -> Transformacion Box-cox

+ Del grafico se observa que la serie no es estacionaria en varianza, por ello realizaremos una transformacion Box-Cox con el valor lambda indicado

In [ ]:
# Compute Box Cox
# Indica si queremos transformacion logaritmica
BOX_COX = True
if BOX_COX:
    # SELECIONAR DE GRAFICO ANTERIOR
    lmbda = 0.1608
    z = st.boxcox(df_ts.values[:,0], lmbda = lmbda) #Convert to positive
    #z,lmbda = st.boxcox(df_ts.values[:,0] - min(df_ts.values) + 1) #Convert to positive and automatic selection of lmbda
    z = pd.DataFrame(z, columns=df_ts.columns.values.tolist())
else:
    z = df_ts

plt.figure(figsize=[15,15])
FT.ts_display(z)

+ Se aprecia levemente que la serie es estacionaria en varinza

In [ ]:
#Check Box Cox of transformed series
FT.boxcox_lambda_plot(z, window_width=12)

+ Tanto el grafico como el valor lambda nos indica que la serie transformada es estacionaria en varianza

# Analyze stationarity

In [ ]:
# Alternative test - Augmented Dickey Fuller Test
# SI P-VALOR MAYOR QUE 0.05 NECESITAS DIFERENCIAR
result = adfuller(z.values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# No estacionaria en media -> Diferenciacion regular

+ p-valor = 0.46 > alpha, por lo tanto no rechazamos la Hipotesis nula, H_0: {Serie no estacionaria} por ello diferenciaremos la serie

In [ ]:
# Difference of the time series
# Regular
d = 1
# Estacional
D = 1
S = 12 # Seasonality of 12 days

Bz = z
for diff in range(d):
    Bz = Bz.diff().dropna() # drop first NA value
for seas_diff in range(D):
    Bz = Bz.diff(S).dropna() # drop first NA values
plt.figure(figsize=[15,15])
FT.ts_display(Bz,lags=50)

+ Tras realizar una diferenciacion regular se puede apreciar que el periodo de la estacionalidad es 12
+ Tras diferenciar estacional y regularmente se siguen observando algunos elementos no modelados correctamente, cerca del valor 100 se podria corresponder a la crisis del año 2008 y cerca del valor 250 al efecto del covid-19, esto se podria solucionar empleando variables impulso (sarimax)

In [ ]:
# Alternative test - Augmented Dickey Fuller Test

# SI P-VALOR MAYOR QUE 0.05 NECESITAS DIFERENCIAR
result = adfuller(Bz.values)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])
print('Critical Values:')
for key, value in result[4].items():
	print('\t%s: %.3f' % (key, value))

# Serie Estacionaria

+ Una vez diferenciada la serie, p-valor < alpha y por lo tanto podemos concluir que la serie transformada y diferenciada es estacionaria

#  Fit SARIMA model

+ Regular: AR(1)
+ Estacional: MA(1)

+ En base a los graficos ACF y PACF de la serie transformada y diferenciada identificamos el modelo.
Para la parte regular nos fijamos el los elementos asociados al perido mas reciente, en este caso, sin tener en cuenta alcunas correlaciones no tan significativas, podemos identificar un AR(1). Para la perte estacionaria, nos fijaremos en las correlaciones asociadas a los periodos, en este caso es claro que se trata de un MA(1). 

In [ ]:
# Fit model with estimated order
sarima_fit = SARIMAX(z, 
                    # D: NUM DIFERENCIACIONES HECHAS
                    order=(1,1,0), # Regular components (p, d, q)
                    seasonal_order=(0, 1, 1, 12), # Seasonal components (p, d, q, s)
                    trend= 'n', # Type of trend: ['c','t','n','ct'] --> [constant, linear, no trend, constant and linear]
                    enforce_invertibility=False, 
                    enforce_stationarity=False).fit()

print(sarima_fit.summary())

# Model analysis

+ AIC =   -252.318, buscamos que este valor sea lo más bajo posible
+ Por otro lado, se observa que todos los coeficientes del modelo son siginificativos
+ Por último, este se trata del modelo SARIMA más simple posible que se adecue al comportamiento de la serie temporal obtenida

# Residuals analysis

In [ ]:
# Plot residual error
plt.figure(figsize=[15,15])
FT.check_residuals(pd.DataFrame(sarima_fit.resid.loc[50:]))

+ Ljung-Box test indica que los residuos son independientes (p-valor > alpha -> No rechazamos H_0:{Residuos independientes})
+ Como ya se comentó, existe comportamiento no modelado en la serie temporal, esto se observa en el plot de los residuos, donde cerca del valor 230 observamos que estos son más significativos (tambien cerca del valor 100 pero mas sutil), así como en los ACF y PACF.
+ Residuos aproximadamente normal, aparecen outliers en la cola derecha, esto se puede deber a comportamiento no modelado u outliers
+ De ACF y PACF se observa que el residuo no es ruido blanco, existen correlaciones significativas -> No se ha modelado todo el comportamiento de la serie temporal

# Predictions

In [ ]:
#Obtain forecasts for in-sample and out-of-sample

# Inicio y horizonte
start = 200
horizon = 20
end = df_ts.shape[0] + horizon

pred = sarima_fit.get_prediction(start=start, end= end, dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure()
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[start:])
plt.plot(yhat)
plt.show()

+ En dicho grafico se observa el valor real como el predicho, se observa además la dificultad del modelo para modelar el efecto del covid-19

In [ ]:
# Plot prediction of out_of_sample and confidence intervals
# If using dynamic = True, the forecast are used as real data
horizon = 20
end = df_ts.shape[0] + horizon

# COMANDO PREDICCION
pred = sarima_fit.get_forecast(steps=horizon, dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure(figsize=[15,15])
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[1000:])
plt.plot(yhat)
plt.show()

# November 2022 prediction

In [ ]:
# Prediccion para steps = 1 (Escala 1 e6)
predPreBC = sarima_fit.get_forecast(steps=1, dynamic=False).predicted_mean
if BOX_COX:
    pred = sp.special.inv_boxcox(predPreBC, lmbda)

print('Prediction:', round(pred.values[0]), 'parados en Noviembre de 2022')

# SARIMAX

+ Emplearemos variables impulso para modelar el comportamiento del Covid-19. Tambien se podrian emplear para modelar el efecto de la crisis del 2008, no obstante el efecto de esta última es modelado de forma más precisa y no supone errores tan significativos

+ Partimos de la serie anterior ya transformada y diferenciada, por tanto estacionaria

# Variables impulso

In [ ]:
# Crear variables impulso
# Observando df_ts: Covid-19: 229-241
# Crecimiento en diferentes etapas, diferentes variables impulso

numExog = 3
exp1 = np.zeros(len(df_ts))
exp2 = np.zeros(len(df_ts))
exp3 = np.zeros(len(df_ts))
exp1[228] = 1
exp2[229] = 1
exp3[230] = 1
exp = np.matrix([exp1, exp2, exp3]).T

+ Debido a que el efecto de la pandemia se fue desarrollando en distintas etapas, cada una de ellas con un impacto diferente, emplearemos tres variables impulso para corregir el modelado de este comportamiento. Solo lo haremos para el inicio de esta, el proceso de estabilización de la serie es modelado de manera mas precisa, no obstante, tambien se podrian emplear estas variables.

# Fit SARIMAX model
Podemos identificar el modelo SARIMAX del mismo modo que se hizo con el SARIMA
+ Regular: AR(1)
+ Estacional: MA(1)

In [ ]:
# Fit model with estimated order
sarimax_fit = SARIMAX(endog = z, 
                    exog = exp,
                    # D: NUM DIFERENCIACIONES HECHAS
                    order=(1,1,0), # Regular components (p, d, q)
                    seasonal_order=(0, 1, 1, 12), # Seasonal components (p, d, q, s)
                    trend= 'n', # Type of trend: ['c','t','n','ct'] --> [constant, linear, no trend, constant and linear]
                    enforce_invertibility=False, 
                    enforce_stationarity=False).fit()

print(sarimax_fit.summary())

# Model analysis

+ AIC = -329.965, buscamos que este valor sea lo más bajo posible
+ Todos los coeficientes del modelo son siginificativos
+ Este se trata del modelo SARIMAX más simple posible que se adecua al comportamiento de la serie temporal
+ Variables impulso permiten modelar el comportamiento del efecto del covid-19

# Residuals analysis

In [ ]:
# Plot residual error
plt.figure(figsize=[15,15])
FT.check_residuals(pd.DataFrame(sarimax_fit.resid.loc[50:]))

+ Ljung-Box test indica que los residuos son independientes (p-valor > alpha -> No rechazamos H_0:{Residuos independientes})
+ Como ya se comentó, existe comportamiento no modelado en la serie temporal, de nuevo, esto se puede observar tanto en el plot de los residuos como en los plot de los ACF y PACF
+ Residuos siguen una distribucion aproximadamente normal
+ De ACF y PACF se observa que el residuo no es ruido blanco, existen correlaciones significativas -> No se ha modelado todo el comportamiento

# Predictions

In [ ]:
#Obtain forecasts for in-sample and out-of-sample
# Inicio y horizonte
start = 200
horizon = 20
end = df_ts.shape[0] + horizon

# Exog variables
exp2Pred = np.concatenate((exp, np.zeros((horizon,numExog))))

pred = sarimax_fit.get_prediction(start=start, end= end, exog= exp[start:start+horizon+1], dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure()
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[start:])
plt.plot(yhat)
plt.show()

+ Se puede observar el valor real de la serie asi como el predicho. Gracias a las variables impulso el efecto inicial de la pandemia es modelado correctamente, algo que tambien ha podido observarse previamente el la grafica de los residuos

In [ ]:
# Plot prediction of out_of_sample and confidence intervals
# If using dynamic = True, the forecast are used as real data
horizon = 20
end = df_ts.shape[0] + horizon

# COMANDO PREDICCION
pred = sarimax_fit.get_forecast(steps=horizon, exog= exp[start:start+horizon], dynamic=False)
yhat = pred.predicted_mean
yhat_conf_int = pred.conf_int(alpha=0.05)

#Undo Box-cox transform if necessary
if BOX_COX:
    yhat = sp.special.inv_boxcox(yhat, lmbda)
    yhat_conf_int = sp.special.inv_boxcox(yhat_conf_int, lmbda)


plt.figure(figsize=[15,15])
plt.fill_between(yhat_conf_int.index,
                yhat_conf_int.iloc[:, 0],
                yhat_conf_int.iloc[:, 1], color='k', alpha=.2)
plt.plot(df_ts.loc[1000:])
plt.plot(yhat)
plt.show()

# November 2022 prediction

In [ ]:
# Prediccion para steps = 1 (Escala 1 e6)
predPreBC = sarimax_fit.get_forecast(steps=1, exog = np.zeros(numExog), dynamic=False).predicted_mean
if BOX_COX:
    pred = sp.special.inv_boxcox(predPreBC, lmbda)

print('Prediction:', round(pred.values[0]), 'parados en Noviembre de 2022')

# Comparacion Modelos

A pesar de ser dos modelos similares existen diferencias entre ellos
+ El valor del AIC es inferior en el caso del SARIMAX, esto se debe a que el efecto inicial de la pandemia (donde en SARIMA se producian errores significativos) es modelado de manera mas precisa y por tanto, disminuye esta valor.
+ En ambos casos se tratan de los modelos mas simples que ajustan la serie temporal donde ademas todos los coeficientes son significativos.
+ En cuanto a los residuos, observando los ACF y PACF podemos determinar que en ambos casos no se trata de ruido blanco, es decir, no se ha modelado todo el comportamineto. En el caso del SARIMAX, podemos ver una reduccion de los residuos asociados al efecto del Covid-19 (gracias a las variables impulso) con respecto a los residuos obtenidos en el SARIMA. Finalmente, en ambos casos son aproximadamente normales, con la diferencia de que los outliers observados en los residuos para el SARIMA (asociados al Covid-19) no se encuentran en el SARIMAX.
+ Por ultimo, en cuanto a las predicciones obtenidas por ambos modelos, estas son muy similares con la diferencia de que SARIMAX ajusta mejor los valores de la fase inicial del covid-19. Las predicciones para Noviembre de 2022 varian aproximadamente en 8000 personas

# Non Linear Model (Time series regression model)

+ Para dicho modelo tomaremos en cuanta unicamente valores lageados de la misma serie (1, 2, 11, 12 y 13 instantes)
+ Habria sido recomendable incorporar otras variables (asi como sus valores lageados) como por ejemplo el valor del PIB en cada mes, por falta de tiempo y recursos no ha sido posible llevarlo a cabo

In [ ]:
# Load aditional data (PIB), shift 1, 2, 11, 12, 13 times Total
dfT = df[['TOTAL']].copy()
dfT['TOTAL_lag1'] = dfT['TOTAL'].shift()
dfT['TOTAL_lag2'] = dfT['TOTAL'].shift(2)
dfT['TOTAL_lag11'] = dfT['TOTAL'].shift(11)
dfT['TOTAL_lag12'] = dfT['TOTAL'].shift(12)
dfT['TOTAL_lag13'] = dfT['TOTAL'].shift(13)

dfT.head()

In [ ]:
# Remove missingnvalues from shifting
dfT.dropna(inplace=True)
dfT.head()

In [ ]:
# Define input and output matrices
OUTPUT = 'TOTAL'
INPUTS = list(dfT.drop(columns = OUTPUT).columns.values)

X = dfT[INPUTS]
y = dfT[OUTPUT]

In [ ]:
# Divide the data into training and test sets sequentialy
# Create random 80/20 split
X_train = X.iloc[0:round(0.8*X.shape[0])]
X_test = X.iloc[round(0.8*X.shape[0])+1:X.shape[0]]
y_train = y.iloc[0:round(0.8*X.shape[0])]
y_test = y.iloc[round(0.8*X.shape[0])+1:X.shape[0]]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

+ Como conjunto de entrenamiento tomaremos el primer 80% de los datos y como test el ultimo 20%. Debido a que no se escogen valores en el rango de tiempo asociado al covid-19, es de esperar que la prediccion para estos valores no sea muy precisa. Para solucionar este problema, se podria tratar de medir dicho efecto con los valores de las varibales impulso obtenidas en SARIMAX (habría que transformarlos pues estos se corresponden a la serie transformada y diferenciada) y tenerlos en cuenta para la prediccion

In [ ]:
# Create dataset to store model predictions
dfTR_eval = X_train.copy()
dfTR_eval['Y'] = y_train.copy() 
dfTS_eval = X_test.copy()
dfTS_eval['Y'] = y_test.copy()

In [ ]:
# MLP Regression Model

# Inputs of the model. 
INPUTS_NUM = INPUTS.copy()
INPUTS_CAT = []
INPUTS_MLP = INPUTS_NUM + INPUTS_CAT

param = {'MLP__alpha': [0.0001,0.001,0.01], # Initial value of regularization
         'MLP__hidden_layer_sizes':[(5,),(13,),(20,),(25,)]} # Number of neurons in each hidden layer, enters as tuples
     
"""
# Uncomment the two following lines for training a single model
param = {'MLP__alpha': [0], # Initial value of regularization
         'MLP__hidden_layer_sizes':[(7,)]} # Number of neurons in each hidden layer, enters as tuples
"""

# Scale data previous to fit and oneHOT
# Prepare numeric variables by scaling values
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
# Prepare the categorical variables by encoding the categories
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore', drop = 'first'))])
# Create a preprocessor to perform the steps defined above
preprocessor = ColumnTransformer(transformers=[
        ('num', numeric_transformer, INPUTS_NUM),
        ('cat', categorical_transformer, INPUTS_CAT)
        ])
pipe = Pipeline(steps=[('preprocessor',preprocessor), # Preprocess the variables when training the model 
                       ('MLP', MLPRegressor(solver='lbfgs', # Update function
                                             activation='logistic', # Logistic sigmoid activation function
                                             #alpha=0.0001, # L2 regularization term
                                             #learning_rate='adaptive', # Type of learning rate used in training
                                             max_iter=4500, # Maximum number of iterations
                                             #batch_size=10, # Size of batch when training
                                             tol=1e-4, # Tolerance for the optimization
                                             #n_iter_no_change=10, # Maximum number of epochs to not meet tol improvement
                                             # random_state=150, # For replication
                                             verbose = True))]) #Print progress

# We use Grid Search Cross Validation to find the best parameter for the model in the grid defined 
nFolds = 10
MLP_fit = GridSearchCV(estimator=pipe, # Structure of the model to use
                       param_grid=param, # Defined grid to search in
                       n_jobs=-1, # Number of cores to use (parallelize)
                       scoring='neg_mean_squared_error', # RMSE 
                       cv=nFolds) # Number of Folds 

# Scale objective variable
outScaler = StandardScaler()
y_train_scaled = outScaler.fit_transform(X = pd.DataFrame(y_train))

MLP_fit.fit(X_train[INPUTS_MLP], y_train_scaled) # Search in grid

+ Para entrenar las redes neuronales es necesario estandarizar los valores objetivos tambien, en otro caso, estas no son capaces de aprender su comportamiento.

In [ ]:
# Plot grid error
MT.plotModelGridError(MLP_fit)

+ Del plot del error anterior podemos deducir la importancia del numero de perceptrones pues a mas numero de estos ultimos, se reduce el error obtenido
+ El valor de alpha parece no ser tan significativo pues los errores obtenidos para cada red son parecidos independientemente del alpha
+ De la combinacion de los dos graficos deducimos que necesita muchos perceptrones y un alpha bajo aunque ninguno de estos en exceso, pues esto llevaria a un sobreentrenamiento como se ve reflejado en ambas graficas produciendose un aumento del error (en el caso de la del alpha se observa ligeramente) 

In [ ]:
# Model Analysis

mlp = MLP_fit.best_estimator_['MLP']
wts = mlp.coefs_
bias = mlp.intercepts_
actfunc = ['identity',MLP_fit.best_estimator_['MLP'].get_params()['activation'],mlp.out_activation_]
X = MLP_fit.best_estimator_['preprocessor'].transform(X_train) # Preprocess the variables
coefnames = X_train.columns.values.tolist()
X = pd.DataFrame(X, columns=coefnames)
y = pd.DataFrame(y_train, columns=['Y'])
sens_end_layer = 'last'
sens_end_input = False
sens_origin_layer = 0
sens_origin_input = True

In [ ]:
sensmlp = ns.jacobian_mlp(wts, bias, actfunc, X, y)

In [ ]:
sensmlp.summary()

In [ ]:
sensmlp.info()

In [ ]:
sensmlp.plot()

+ De las graficas obtenidas se puede identificar que variables (valores retardados) son mas significativos para la predicción del siguiente instante

In [ ]:
# Obtain a report of the model based on predictions
# Rescale output

dfTR_eval['MLP_pred'] = outScaler.inverse_transform(pd.DataFrame(MLP_fit.predict(X_train)))
dfTS_eval['MLP_pred'] = outScaler.inverse_transform(pd.DataFrame(MLP_fit.predict(X_test)))

In [ ]:
#Training and test errors

print('Training MAE:',mean_absolute_error(dfTR_eval['Y'], dfTR_eval['MLP_pred']))
print('Test MAE:',mean_absolute_error(dfTS_eval['Y'], dfTS_eval['MLP_pred']))

print('Training RMSE:',math.sqrt(mean_squared_error(dfTR_eval['Y'], dfTR_eval['MLP_pred'])))
print('Test RMSE:',math.sqrt(mean_squared_error(dfTS_eval['Y'], dfTS_eval['MLP_pred'])))

print('Training R2:',r2_score(dfTR_eval['Y'], dfTR_eval['MLP_pred']))
print('Test R2:',r2_score(dfTS_eval['Y'], dfTS_eval['MLP_pred']))

+ Observamos diferencia entre train y test, se puede deber a sobreentrenamiento o al efecto del covid-19 que como es de esperar, no esta modelado de forma precisa
+ Diferencia entre MAE y RMSE en test, puede deberse a ouliers, es probable que se correspondan a los valores asociados al covid-19
+ El modelo explica de forma correcta la varianza de la variable objetivo

In [ ]:
# Plot predictions of the model

sns.scatterplot(x='TOTAL_lag1', y='Y', data=dfTR_eval, color='black', alpha=0.5)
sns.scatterplot(x='TOTAL_lag1', y='MLP_pred', data=dfTR_eval, color='red', edgecolor='black').set_title('Predictions for training data')
plt.show()

+ Ademas de la precision de las predicciones se puede observar la correlacion entre la variable objetivo y su valor un instante retardado 

In [ ]:
# Analysis of residuals

RT.plotModelDiagnosis(dfTS_eval, 'MLP_pred', 'Y', figsize = (15,15))

+ En cuanto a los residuos, observamos que oscilan en torno al 0 pero este valor no se mantiene constante en practicamente ningun grafico, identificamos tambien la presencia de outliers, esto es un indicador de que exite gran parte de comportamiento no modelado
+ Lo ideal seria esperar que los errores siguiesen una normal de media 0 y varianza constante

In [ ]:
# Visualize prediction against real value
sns.scatterplot(data=dfTS_eval, x='Y', y='Y', color='black', alpha=0.5)
sns.scatterplot(data=dfTS_eval, x='MLP_pred', y='Y', color='blue', edgecolor='black')
plt.xlabel('Predicted')
plt.show()

+ Valores reales frente a los predichos, de nuevo se observan ouliers, probablemente debidos al efecto del covid-19

In [ ]:
# Compare forecasts in time

plt.figure()
plt.plot('Y', data=dfTS_eval, label='Real')
plt.plot('MLP_pred', data=dfTS_eval, label='Forecast')
plt.legend()
plt.show()

+ Se observan los valores reales de la serie así como su prediccion

In [ ]:
# Check correlation of residuals

plt.figure(figsize = [15, 15])
FT.ts_display(dfTS_eval['Y'] - dfTS_eval['MLP_pred'], lags = 23)

+ Errores covid-19, no en entrenamiento, no esperado
+ Residuos no ruido blanco, comportamiento no modelado
+ Como ya hemos comentado, se podrian emplear variables como PIB

# Final Prediction November 2022

In [ ]:
lastValue = df.index[-1]
dataPred = X_test.loc[lastValue:, :]
valuePred = outScaler.inverse_transform(pd.DataFrame(MLP_fit.predict(dataPred)))
print('Prediction:', round(valuePred[0][0]), 'parados en Noviembre de 2022')

+ La prediccion obtenida desentona mas con las anteriores, ofrece un valor mas alto, aproximadamente 70000 personas

# Prediccion final

+ Basada en SARIMA, SARIMAX o combinacion de estas (e.g. media aritmetica)